In [ ]:
"""Run two below lines to get my_abc module"""
# !git clone https://github.com/thanhttttt/thanh.git
# !pip install -r /content/thanh/requirements.txt
"""Run two below lines to drive"""
# from google.colab import drive
# drive.mount('/content/drive')

# Import lib

In [ ]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import Sionna
import sys
sys.path.append('../')
sys.path.append('/content/thanh/')
sys.path.append('../thanh/')
import sionna

# try:
#     import sionna
# except ImportError as e:
#     # Install Sionna if package is not already installed
#     import os
#     os.system("pip install sionna")
#     import sionna

import tensorflow as tf
# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

sionna.config.seed = 42 # Set seed for reproducible results

# Load the required Sionna components
from sionna.nr.my_abc import *

# Gen function (Custom if need)

In [ ]:
"""Uncomment and customize if needed; otherwise, the default function from `my_abc` will be used."""

# def generate_data(name: str,
#         data_dir: str,
#         pusch_configs: List[MyPUSCHConfig],
#         channel_scenarios: List[str],
#         esno_dbs: List[float],
#         slots: List[int],
#         save_dataset: str = None):
#     assert save_dataset in [None, 'hdf5', 'pickle']
#     """set up save directory"""
#     pusch_records=[]
#     parquet_dir = f'{data_dir}/parquet'
#     if save_dataset: os.makedirs(parquet_dir, exist_ok=True)
#     hdf5_dir = f'{data_dir}/hdf5'
#     if save_dataset == 'hdf5': os.makedirs(hdf5_dir, exist_ok=True)
#     pickle_dir = f'{data_dir}/pickle'
#     if save_dataset == 'pickle': os.makedirs(pickle_dir, exist_ok=True)

#     """set up for per slot"""
#     len_per_case = len(slots)

#     total_iterations = len(pusch_configs) * len(esno_dbs) * len_per_case


#     """generate ..."""
#     with tqdm(total=total_iterations, desc="Generating Data") as pbar:
#         for config_idx, pusch_config in enumerate(pusch_configs):
           
#             Pusch_Pilots = {}
#             Pusch_Slots = set(slots)
#             for n, slot in enumerate(Pusch_Slots):
#                 pusch_config_i = pusch_config.clone()
#                 pusch_config_i.carrier.slot_number = slot
#                 pilot_pattern_i = PUSCHPilotPattern([pusch_config_i], dtype=tf.complex64)
#                 Pusch_Pilots[slot] = pilot_pattern_i.pilots
#             len_pusch = len(Pusch_Slots)

#             """set up for all case"""
#             carrier_frequency = pusch_config.My_Config.Carrier_frequency

#             simulator = MySimulator(pusch_config)

#             channel_scenario = random.choice(channel_scenarios)
#             """channel_scenario form: {CDL}-{A|B|C|D|E}-{delay_spread (ns)}-{speed (m/s)}
#                                 or {Umi|Uma}-{low|high}-[OnPL]-[OnSF]-{delay_spread (ns)}-{speed (m/s)}

#                 Ex: CDL-A-150-10"""

#             chn_scn = channel_scenario.split('-')
            
#             model = chn_scn[1]
#             channel = chn_scn[0]
#             enable_pl = True if 'OnPL' in chn_scn else False # Umi/Uma enable pathloss
#             enable_sf = True if 'OnSF' in chn_scn else False # Umi/Uma enable shadow fading

#             speed = float(chn_scn[-1])
#             delay_spread = float(chn_scn[-2])
            

#             if 'CDL' == channel:
#                 channel_model = CDL(model = model,
#                                         delay_spread = delay_spread*1e-9,
#                                         carrier_frequency = carrier_frequency,
#                                         ut_array = ue_antenna,
#                                         bs_array = gnb_array,
#                                         direction = "uplink",
#                                         min_speed = speed,
#                                         max_speed = speed)

#             else:
#                 if 'Umi' == channel:
#                     channel_model = UMi(carrier_frequency = carrier_frequency,
#                                         o2i_model = model,
#                                         ut_array = ue_antenna,
#                                         bs_array = gnb_array,
#                                         direction = "uplink",
#                                         enable_pathloss = enable_pl,
#                                         enable_shadow_fading = enable_sf)
#                 elif 'Uma' == channel:
#                     channel_model = UMa(carrier_frequency = carrier_frequency,
#                                         o2i_model = model,
#                                         ut_array = ue_antenna,
#                                         bs_array = gnb_array,
#                                         direction = "uplink",
#                                         enable_pathloss = enable_pl,
#                                         enable_shadow_fading = enable_sf)

#             simulator = MySimulator(pusch_config)

#             channel_i = OFDMChannel(channel_model=channel_model, resource_grid=simulator.resource_grid,
#                                     add_awgn=False, normalize_channel=True, return_channel=True)
            
#             for esno_db in esno_dbs:
#                 no_scaling = pow(10., -esno_db / 10.)
#                 if channel in ['Umi', 'Uma']:
#                     channel_i._cir_sampler.set_topology(*gen_topology(1,1,channel.lower(),min_ut_velocity=speed, max_ut_velocity=speed))

#                 for n,slot in enumerate(slots):
#                     status_str = f"(config {config_idx} | channel {channel_scenario} | {esno_db} dB | slot {slot} | Sample: {n+1}/{len_per_case}"
#                     pbar.set_description(status_str)
#                     simulator.pusch_config.carrier.slot_number = slot
#                     simulator.update_pilots(Pusch_Pilots[slot])
                    
#                     b, c, y= simulator.sim(1, channel_i, no_scaling, return_channel=False)

#                     assert b.shape[0] == b.shape[1] == c.shape[0] == c.shape[1] == y.shape[0] == y.shape[1] == 1
#                     b = tf.cast(b, dtype=tf.uint8)[0][0]
#                     c = tf.cast(c, dtype=tf.uint8)[0][0]
#                     r = tf.transpose(tf.constant(simulator.pusch_config.dmrs_grid, dtype=y.dtype), [0, 2, 1])
#                     y =  y[0][0]

#                     timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")


#                     # print(y, r)
#                     if save_dataset == 'hdf5':
#                         save_hdf5([b,c,y,r], f'{hdf5_dir}/{name}', timestamp)
#                     if save_dataset == 'pickle':
#                         save_pickle([b,c,y,r], f'{pickle_dir}/{name}', timestamp)

                    
                    
#                     pusch_records.append(PuschRecord(
#                                         nPhyCellId=pusch_config.carrier.n_cell_id,
#                                         nSFN=(n // len_pusch) % 1023,
#                                         nSlot=slot,
#                                         nPDU=1,
#                                         nGroup=1,
#                                         nUlsch=1,
#                                         nUlcch=0,
#                                         nRachPresent=0,
#                                         nRNTI=pusch_config.n_rnti,
#                                         nUEId=0,
#                                         nBWPSize=pusch_config.n_size_bwp,
#                                         nBWPStart=pusch_config.n_start_bwp,
#                                         nSubcSpacing=pusch_config.carrier.mu,
#                                         nCpType=pusch_config.My_Config.Sys.CpType,
#                                         nULType=0,
#                                         nMcsTable=pusch_config.tb.mcs_table - 1,
#                                         nMCS=pusch_config.tb.mcs_index,
#                                         nTransPrecode=pusch_config.My_Config.Ue[0].TransformPrecoding,
#                                         nTransmissionScheme=pusch_config.My_Config.Ue[0].CodeBookBased,
#                                         nNrOfLayers=pusch_config.num_layers,
#                                         nPortIndex=pusch_config.dmrs.dmrs_port_set,
#                                         nNid=pusch_config.tb.n_id,
#                                         nSCID=pusch_config.dmrs.n_scid,
#                                         nNIDnSCID=pusch_config.dmrs.n_id[0],
#                                         nNrOfAntennaPorts=pusch_config.My_Config.Sys.NRxAnt,
#                                         nVRBtoPRB=0,
#                                         nPMI=pusch_config.My_Config.Ue[0].Tpmi,
#                                         nStartSymbolIndex=pusch_config.symbol_allocation[0],
#                                         nNrOfSymbols=pusch_config.symbol_allocation[1],
#                                         nResourceAllocType=1,
#                                         nDMRSTypeAPos=pusch_config.dmrs.type_a_position,
#                                         nRBStart=pusch_config.first_resource_block,
#                                         nRBSize=pusch_config.num_resource_blocks,
#                                         nTBSize=(pusch_config.tb_size//8),
#                                         nRV=pusch_config.My_Config.Sys.rvSeq,
#                                         nHARQID=n % 16,
#                                         nNDI=1,
#                                         nMappingType=pusch_config.My_Config.Ue[0].PuschMappingType,
#                                         nDMRSConfigType=pusch_config.My_Config.Ue[0].DmrsConfigurationType,
#                                         nNrOfCDMs=pusch_config.dmrs.num_cdm_groups_without_data,
#                                         nNrOfDMRSSymbols=pusch_config.dmrs.length,
#                                         nDMRSAddPos=pusch_config.dmrs.additional_position,
#                                         nPTRSPresent=pusch_config.My_Config.Ue[0].Ptrs,
#                                         nAck=pusch_config.My_Config.Ue[0].OAck,
#                                         nAlphaScaling=pusch_config.My_Config.Ue[0].ScalingFactor,
#                                         nBetaOffsetACKIndex=pusch_config.My_Config.Ue[0].IHarqAckOffset,
#                                         nCsiPart1=pusch_config.My_Config.Ue[0].OCsi1,
#                                         nBetaOffsetCsiPart1Index=pusch_config.My_Config.Ue[0].ICsi1Offset,
#                                         nCsiPart2=pusch_config.My_Config.Ue[0].OCsi2,
#                                         nBetaOffsetCsiPart2Index=pusch_config.My_Config.Ue[0].ICsi2Offset,
#                                         nTpPi2BPSK=pusch_config.My_Config.Ue[0].TpPi2Bpsk,
#                                         nTPPuschID=pusch_config.My_Config.Ue[0].NRsId,
#                                         nRxRUIdx=np.arange(0, pusch_config.My_Config.Sys.NRxAnt),
#                                         nUE=1,
#                                         nPduIdx=[0],
#                                         Channel_model=f"{channel}-{model}",
#                                         Speed=speed,
#                                         Delay_spread=delay_spread,
#                                         Esno_db=esno_db,
#                                         Data_filename=timestamp,
#                                         Data_dirname=name
#                                     )
#                                 )
#                     pbar.update(1)  # Increment progress

#             df = pd.DataFrame.from_records(pusch_records, columns=PuschRecord._fields)
#             if save_dataset: df.to_parquet(f'{parquet_dir}/{name}.parquet', engine="pyarrow")
#     return df

# Config case to gen...

In [33]:
"""Channel config"""
"""Each sample will randomly select one channel scenario from the following list."""
channel_scenarios = [
    'CDL-A-150-1',
    'CDL-B-150-1',
    'CDL-C-150-1',
    'CDL-D-150-1',
    'CDL-E-150-1',
    'CDL-A-10-10',
    'CDL-B-10-10',
    'CDL-C-10-10',
    'CDL-D-10-10',
    'CDL-E-10-10',
    'CDL-A-50-4',
    'CDL-B-50-4',
    'CDL-C-50-4',
    'CDL-D-50-4',
    'CDL-E-50-4'
]
"""Each sample will sequentially iterate through the following Es/No values (dB)"""
esno_dbs = [i for i in np.arange(start=5.,stop=-5.1,step=-0.5)]

In [42]:
"""Pusch config"""
"""NRB (Number of Resource Blocks) and MCS (Modulation and Coding Scheme) are fixed across all samples."""
NRB = 130
MCS = 8
"""Prototype configuration generation for system and UE settings"""
sysCfg = SystemConfig(
                    NCellId = 442,
                    FrequencyRange = 1,
                    BandWidth = 60,
                    Numerology = 1,
                    CpType = 0,
                    NTxAnt = 1,
                    NRxAnt = 8,
                    BwpNRb = 162,
                    BwpRbOffset = 0,
                    harqProcFlag = 0,
                    nHarqProc = 1,
                    rvSeq = 0
                )
ueCfg = UeConfig(
                TransformPrecoding = 0,
                Rnti = 40035,
                nId = 442,
                CodeBookBased = 0,
                DmrsPortSetIdx = [0],
                NLayers = 1,
                NumDmrsCdmGroupsWithoutData = 2,
                Tpmi = 0,
                FirstSymb = 0,
                NPuschSymbAll = 14,
                RaType = 1,
                FirstPrb = 0,
                NPrb = NRB,
                FrequencyHoppingMode = 0,
                McsTable = 0,
                Mcs = MCS,
                ILbrm = 0,
                nScId = 0,
                NnScIdId = 442,
                DmrsConfigurationType = 0,
                DmrsDuration = 1,
                DmrsAdditionalPosition = 1,
                PuschMappingType = 0,
                DmrsTypeAPosition = 3,
                HoppingMode = 0,
                NRsId = 0,
                Ptrs = 0,
                ScalingFactor = 0,
                OAck = 0,
                IHarqAckOffset = 11,
                OCsi1 = 0,
                ICsi1Offset = 7,
                OCsi2 = 0,
                ICsi2Offset = 0,
                NPrbOh = 0,
                nCw = 1,
                TpPi2Bpsk = 0
            )
myCfg = MyConfig(sysCfg, [ueCfg])
puschCfg = MyPUSCHConfig(myCfg)
# puschCfg.show() # uncomment for detail

In [ ]:
"""Generate [num_cfgs] random configuration cases for (PCI, RNTI, RB_START)"""
num_cfgs = 40 
random.seed(42)
# pci_values = random.sample(range(1024), num_cfgs)
pci_values = random.choices(range(1008), k=num_cfgs)  # Allows duplicates# pci_values = range(1008)

rnti_values = [random.randint(0, 65535) for _ in range(num_cfgs)]
rbstart_values = [random.randint(0, puschCfg.carrier.n_size_grid - NRB) for _ in range(num_cfgs)]
puschCfg_set = []
# abc = []
for pci, rnti, rbstart in zip(pci_values, rnti_values, rbstart_values):
    pusch_instance = puschCfg.clone()
    pusch_instance.phy_cell_id = pci
    pusch_instance.n_rnti = rnti
    pusch_instance.My_Config.Ue[0].FirstPrb = rbstart
    puschCfg_set.append(pusch_instance)
    # abc.append((pci, rnti, rbstart))

In [43]:
"""Generate data"""
# data_dir = '/content/dataset' # if colab
data_dir = '../Pusch_data/dataset' # if workspace

# You can manually set a custom name if needed. Example: name = "4RB_dataset"
name = datetime.now().strftime("%Y%m%d%H%M%S%f")  

start = time.time()

df = generate_data(
    name=f'{name}',
    data_dir=data_dir,
    pusch_configs=[puschCfg, *puschCfg_set],
    channel_scenarios=channel_scenarios,
    esno_dbs=esno_dbs,
    slots=[4, 5, 14, 15],  # Slots per configuration; Use slots=[slot_id] for a single slot type; Repeat with slots=[slot_id_0,...] * n
    save_dataset=None  # 'hdf5' if saving to hdf5 dataset; Set to None if saving is not required; 
)

duration = time.time() - start

(config 0 | channel CDL-D-10-10 | 3.5 dB | slot 15 | Sample: 4/4:   0%|          | 15/3444 [00:10<41:36,  1.37it/s] 


KeyboardInterrupt: 